In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, QuantileTransformer
import xgboost as xgb
import numpy as np

In [1]:
# Carregar o dataset
df = pd.read_csv('./datasets/casas_ponta_grossa.csv', delimiter=',')

# Retirando valores NaN:
df = df[df['Quartos'].notna()]
df = df[df['Banheiros'].notna()]
df = df[df['Suítes'].notna()]
df = df[df['Vagas de Garagem'].notna()]
df = df[df['Bairro'].notna()]
df = df[df['Área Total'].notna()]
dfFiltrado = df[df['Valor Venda'].notna()]

# Filtrando as casas com valor de venda entre 1 milhão e 5 milhões
dfFiltrado = dfFiltrado[(dfFiltrado['Valor Venda'] >= 1000000) & (dfFiltrado['Valor Venda'] <= 5000000)]

# Engenharia de Features
# Transformando a coluna 'Bairro' com LabelEncoder
le = LabelEncoder()
dfFiltrado['Bairro'] = le.fit_transform(dfFiltrado['Bairro'])

# Normalizando as variáveis numéricas
scaler = QuantileTransformer(output_distribution='normal')  # Usando QuantileTransformer para robustez
dfFiltrado[['Área Total', 'Quartos', 'Banheiros', 'Suítes', 'Vagas de Garagem']] = scaler.fit_transform(
    dfFiltrado[['Área Total', 'Quartos', 'Banheiros', 'Suítes', 'Vagas de Garagem']]
)

# Criando novas colunas
dfFiltrado['Preco por m2'] = dfFiltrado['Valor Venda'] / dfFiltrado['Área Total']
dfFiltrado['Faixa de Preço'] = pd.cut(dfFiltrado['Valor Venda'], bins=[1000000, 2000000, 3500000, 5000000], labels=['Baixo', 'Médio', 'Alto'])
dfFiltrado['Quartos por m2'] = dfFiltrado['Quartos'] / dfFiltrado['Área Total']
dfFiltrado['Banheiros por m2'] = dfFiltrado['Banheiros'] / dfFiltrado['Área Total']
dfFiltrado['Vagas de Garagem por m2'] = dfFiltrado['Vagas de Garagem'] / dfFiltrado['Área Total']

# Novas colunas sugeridas
dfFiltrado['Área Total * Quartos'] = dfFiltrado['Área Total'] * dfFiltrado['Quartos']
dfFiltrado['Área Total * Banheiros'] = dfFiltrado['Área Total'] * dfFiltrado['Banheiros']
dfFiltrado['Diferença Preço - Média Bairro'] = dfFiltrado['Valor Venda'] - dfFiltrado.groupby('Bairro')['Valor Venda'].transform('mean')
dfFiltrado['Bairro por Faixa de Preço'] = dfFiltrado['Bairro'] * dfFiltrado['Faixa de Preço'].cat.codes
dfFiltrado['Área por Quartos'] = dfFiltrado['Área Total'] / dfFiltrado['Quartos']
dfFiltrado['Área por Banheiros'] = dfFiltrado['Área Total'] / dfFiltrado['Banheiros']

# Convertendo a coluna 'Faixa de Preço' para valores numéricos (inteiros)
dfFiltrado['Faixa de Preço'] = dfFiltrado['Faixa de Preço'].cat.codes

# Verificação da coerência dos dados
# Verificar tipos de dados e valores nulos
print(dfFiltrado.info())

# Visualizar as primeiras linhas do dataset
print(dfFiltrado.head())


NameError: name 'pd' is not defined